<a href="https://colab.research.google.com/github/mahdi-alalawi/AndroidTutorialForBeginners/blob/master/%D8%AA%D8%B9%D8%AF%D9%8A%D9%84%D8%A7%D8%AA_%D8%B9_%D8%A7%D9%84%D8%B1%D8%B3%D9%88%D9%85%D8%A7%D8%AA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.pyplot as plt

def plot_bollinger_bands(data, window=20, num_std_dev=2):
    """رسم بولنجر باندز على مخطط بياني."""
    # حساب المتوسط المتحرك والانحراف المعياري
    data['Middle_Band'] = data['close'].rolling(window=window).mean()
    data['Std_Dev'] = data['close'].rolling(window=window).std()

    # حساب الحد العلوي والحد السفلي
    data['Upper_Band'] = data['Middle_Band'] + (data['Std_Dev'] * num_std_dev)
    data['Lower_Band'] = data['Middle_Band'] - (data['Std_Dev'] * num_std_dev)

    # طباعة القيم
    print("Middle Band:", data['Middle_Band'].iloc[-1])
    print("Upper Band:", data['Upper_Band'].iloc[-1])
    print("Lower Band:", data['Lower_Band'].iloc[-1])

    # رسم المخطط
    plt.figure(figsize=(12, 6))
    plt.plot(data['close'], label='Close Price', color='blue')
    plt.plot(data['Middle_Band'], label='Middle Band', color='orange')
    plt.plot(data['Upper_Band'], label='Upper Band', color='green')
    plt.plot(data['Lower_Band'], label='Lower Band', color='red')
    plt.fill_between(data.index, data['Upper_Band'], data['Lower_Band'], color='lightgray', alpha=0.5)
    plt.title('Bollinger Bands')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

# مثال على كيفية استخدام الدالة
# تأكد من أن لديك بيانات في DataFrame باسم `data` تحتوي على عمود 'close'
# plot_bollinger_bands(data)


In [ ]:
import matplotlib.pyplot as plt

def plot_premium_discount(data):
    """رسم Premium وDiscount على مخطط بياني."""
    # تأكد من أن البيانات تحتوي على الأعمدة المطلوبة
    data['Premium'] = data['close'] - data['low'].rolling(window=20).min()
    data['Discount'] = data['high'].rolling(window=20).max() - data['close']

    # طباعة القيم الأخيرة
    print("Last Premium Value:", data['Premium'].iloc[-1])
    print("Last Discount Value:", data['Discount'].iloc[-1])

    # رسم المخطط
    plt.figure(figsize=(12, 6))
    plt.plot(data.index, data['Premium'], label='Premium', color='blue')
    plt.plot(data.index, data['Discount'], label='Discount', color='red')
    plt.fill_between(data.index, data['Premium'], color='lightblue', alpha=0.5)
    plt.fill_between(data.index, data['Discount'], color='lightcoral', alpha=0.5)
    plt.title('Premium and Discount Bands')
    plt.xlabel('Date')
    plt.ylabel('Value')
    plt.legend()
    plt.show()

# مثال على كيفية استخدام الدالة
# تأكد من أن لديك بيانات في DataFrame باسم `data` تحتوي على الأعمدة 'close', 'low', 'high'
# plot_premium_discount(data)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

def calculate_rsi(data, window=14):
    """حساب مؤشر القوة النسبية (RSI)."""
    delta = data['close'].diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()

    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))

    return rsi

def plot_rsi(data, window=14):
    """رسم مؤشر القوة النسبية (RSI) على مخطط بياني."""
    data['RSI'] = calculate_rsi(data, window)

    # طباعة القيمة الأخيرة لـ RSI
    print("Last RSI Value:", data['RSI'].iloc[-1])

    # رسم المخطط
    plt.figure(figsize=(12, 6))
    plt.plot(data.index, data['RSI'], label='RSI', color='blue')
    plt.axhline(70, linestyle='--', color='red', label='Overbought (70)')
    plt.axhline(30, linestyle='--', color='green', label='Oversold (30)')
    plt.title('Relative Strength Index (RSI)')
    plt.xlabel('Date')
    plt.ylabel('RSI Value')
    plt.legend()
    plt.ylim(0, 100)
    plt.show()

# مثال على كيفية استخدام الدالة
# تأكد من أن لديك بيانات في DataFrame باسم `data` تحتوي على عمود 'close'
# plot_rsi(data)
